In [1]:
%pip uninstall --yes 'keras' 'matplotlib' 'scikit-learn' 'tensorflow'

Found existing installation: keras 3.10.0
Uninstalling keras-3.10.0:
  Successfully uninstalled keras-3.10.0
Found existing installation: matplotlib 3.10.0
Uninstalling matplotlib-3.10.0:
  Successfully uninstalled matplotlib-3.10.0
Found existing installation: scikit-learn 1.6.1
Uninstalling scikit-learn-1.6.1:
  Successfully uninstalled scikit-learn-1.6.1
Found existing installation: tensorflow 2.19.0
Uninstalling tensorflow-2.19.0:
  Successfully uninstalled tensorflow-2.19.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
import warnings
warnings.simplefilter('ignore')

In [3]:
import os
import sys
import subprocess

In [4]:
def set_env(input_archive, temp_dir):

    if not os.path.exists(temp_dir):
        os.makedirs(temp_dir, exist_ok=True)
        
        subprocess.run(['tar', '-xzf', input_archive, '-C', temp_dir], check=True)
    
    subprocess.run([
        sys.executable, 
        '-m', 
        'pip', 
        'install', 
        '--no-index', 
        '--find-links', 
        f'{temp_dir}/wheels', 
        'unsloth', 
        'trl', 
        'vllm', 
        'openai_harmony'
    ], check=True)

In [5]:
set_env(
    input_archive='/kaggle/input/aimo-3-utils/wheels.tar.gz', 
    temp_dir='/kaggle/tmp/setup'
)

Looking in links: /kaggle/tmp/setup/wheels
Processing /kaggle/tmp/setup/wheels/unsloth-2025.12.9-py3-none-any.whl
Processing /kaggle/tmp/setup/wheels/trl-0.24.0-py3-none-any.whl
Processing /kaggle/tmp/setup/wheels/vllm-0.11.2-cp38-abi3-manylinux1_x86_64.whl
Processing /kaggle/tmp/setup/wheels/openai_harmony-0.0.8-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/tmp/setup/wheels/unsloth_zoo-2025.12.7-py3-none-any.whl (from unsloth)
Processing /kaggle/tmp/setup/wheels/tyro-1.0.3-py3-none-any.whl (from unsloth)
Processing /kaggle/tmp/setup/wheels/xformers-0.0.33.post1-cp39-abi3-manylinux_2_28_x86_64.whl (from unsloth)
Processing /kaggle/tmp/setup/wheels/bitsandbytes-0.49.0-py3-none-manylinux_2_24_x86_64.whl (from unsloth)
Processing /kaggle/tmp/setup/wheels/datasets-4.3.0-py3-none-any.whl (from unsloth)
Processing /kaggle/tmp/setup/wheels/prometheus_fastapi_instrumentator-7.1.0-py3-none-any.whl (from vllm)
Processing /kaggle/tmp/setup/wheels/lm_format_enforcer-0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kauldron 1.3.0 requires scikit-learn, which is not installed.
kauldron 1.3.0 requires tensorflow, which is not installed.
ydata-profiling 4.18.0 requires matplotlib<=3.10,>=3.5, which is not installed.
pyldavis 3.4.1 requires scikit-learn>=1.0.0, which is not installed.
stable-baselines3 2.1.0 requires matplotlib, which is not installed.
sentence-transformers 5.1.1 requires scikit-learn, which is not installed.
librosa 0.11.0 requires scikit-learn>=1.1.0, which is not installed.
cuml-cu12 25.6.0 requires scikit-learn>=1.5, which is not installed.
bigframes 2.26.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
bigframes 2.26.0 requires matplotlib>=3.7.1, which is not installed.
arviz 0.22.0 requires matplotlib>=3.8, which is not installed.
pynndescent 0.5.13 requires scikit-learn>=0.

In [6]:
subprocess.run(['ls', '/kaggle/tmp/setup/tiktoken_encodings'])

cl100k_base.tiktoken
o200k_base.tiktoken


CompletedProcess(args=['ls', '/kaggle/tmp/setup/tiktoken_encodings'], returncode=0)

In [7]:
os.environ['TRANSFORMERS_NO_TF'] = '1'
os.environ['TRANSFORMERS_NO_FLAX'] = '1'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
os.environ['TRITON_PTXAS_PATH'] = '/usr/local/cuda/bin/ptxas'
os.environ['TIKTOKEN_ENCODINGS_BASE'] = '/kaggle/tmp/setup/tiktoken_encodings'

In [8]:
import gc
import re
import math
import time
import queue
import threading
import contextlib
from typing import Optional
from jupyter_client import KernelManager
from collections import Counter, defaultdict
from concurrent.futures import as_completed, ThreadPoolExecutor

import pandas as pd
import polars as pl

from openai import OpenAI

from openai_harmony import (
    HarmonyEncodingName, 
    load_harmony_encoding, 
    SystemContent, 
    ReasoningEffort, 
    ToolNamespaceConfig, 
    Author, 
    Message, 
    Role, 
    TextContent, 
    Conversation
)

from transformers import set_seed
import kaggle_evaluation.aimo_3_inference_server

In [ ]:
class CFG:

    system_prompt = (
        'You are a world-class International Mathematical Olympiad (IMO) competitor. '
        'The final answer must be a non-negative integer between 0 and 99999. '
        'You must place the final integer answer inside \\boxed{}.'
    )
    
    tool_prompt = (
        'Use this tool to execute Python code. '
        'The environment is a stateful Jupyter notebook. '
        'You must use print() to output results.'
    )

    preference_prompt = (
        'Use `math`, `numpy`,`sympy`,`itertools` and `collections` to solve the problem.'
    )

    served_model_name = 'gpt-oss'
    model_path = '/kaggle/input/gpt-oss-120b/transformers/default/1'
    
    kv_cache_dtype = 'fp8_e4m3'
    dtype = 'auto'

    high_problem_timeout = 900
    base_problem_timeout = 300

    notebook_limit = 17520
    server_timeout = 180

    session_timeout = 960
    jupyter_timeout = 10
    sandbox_timeout = 5

    stream_interval = 200
    context_tokens = 65536
    search_tokens = 1024
    buffer_tokens = 512
    batch_size = 256
    early_stop = 4
    attempts = 8
    workers = 16
    turns = 128
    seed = 42

    gpu_memory_utilization = 0.96
    temperature = 1.0
    min_p = 0.02

    # Adaptive retry config
    retry_system_prompt = (
        'You are a world-class International Mathematical Olympiad (IMO) competitor. '
        'The final answer must be a non-negative integer between 0 and 99999. '
        'You must place the final integer answer inside \\boxed{}. '
        'Think carefully and try a different approach than what seems obvious. '
        'If a computational approach seems natural, try analytical reasoning first. '
        'Verify your answer by checking special cases or substituting back.'
    )
    retry_attempts = 4
    min_consensus = 2
    min_retry_time = 60

    # Time banking config
    max_bank_bonus = 300

In [10]:
set_seed(CFG.seed)

In [11]:
class AIMO3Template:

    def __init__(self):

        pass

    def get_system_content(self, system_prompt: str, tool_config: ToolNamespaceConfig) -> SystemContent:

        return (
            SystemContent.new()
            .with_model_identity(system_prompt)
            .with_reasoning_effort(reasoning_effort=ReasoningEffort.HIGH)
            .with_tools(tool_config)
        )

    def apply_chat_template(
        self, 
        system_prompt: str, 
        user_prompt: str, 
        tool_config: ToolNamespaceConfig
    ) -> list[Message]:

        system_content = self.get_system_content(system_prompt, tool_config)        
        system_message = Message.from_role_and_content(Role.SYSTEM, system_content)

        user_message = Message.from_role_and_content(Role.USER, user_prompt)

        return [system_message, user_message]

In [12]:
class AIMO3Sandbox:

    _port_lock = threading.Lock()
    _next_port = 50000

    @classmethod
    def _get_next_ports(cls, count: int = 5) -> list[int]:

        with cls._port_lock:
            ports = list(range(cls._next_port, cls._next_port + count))
            cls._next_port += count

            return ports

    def __init__(self, timeout: float):

        self._default_timeout = timeout
        self._owns_kernel = False
        self._client = None
        self._km = None
        
        ports = self._get_next_ports(5)

        env = os.environ.copy()
        env['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'
        env['PYDEVD_WARN_EVALUATION_TIMEOUT'] = '0'
        env['JUPYTER_PLATFORM_DIRS'] = '1'
        env['PYTHONWARNINGS'] = 'ignore'
        env['MPLBACKEND'] = 'Agg'

        self._km = KernelManager()
        self._km.shell_port = ports[0]
        self._km.iopub_port = ports[1]
        self._km.stdin_port = ports[2]
        self._km.hb_port = ports[3]
        self._km.control_port = ports[4]

        self._km.start_kernel(env=env, extra_arguments=['--Application.log_level=CRITICAL'])

        self._client = self._km.blocking_client()
        self._client.start_channels()
        self._client.wait_for_ready(timeout=self._default_timeout)
        self._owns_kernel = True

        self.execute(
            'import math\n'
            'import sympy\n'
            'import itertools\n'
            'import collections\n'
            'import numpy as np\n'
        )

    def _format_error(self, traceback: list[str]) -> str:

        clean_lines = []

        for frame in traceback:
            clean_frame = re.sub(r'\x1b\[[0-9;]*m', '', frame)

            if 'File "' in clean_frame and 'ipython-input' not in clean_frame:
                continue

            clean_lines.append(clean_frame)

        return ''.join(clean_lines)

    def execute(self, code: str, timeout: float | None = None) -> str:

        client = self._client
        effective_timeout = timeout or self._default_timeout
        
        msg_id = client.execute(
            code, 
            store_history=True, 
            allow_stdin=False, 
            stop_on_error=False
        )

        stdout_parts = []
        stderr_parts = []
        
        start_time = time.time()

        while True:
            elapsed = time.time() - start_time

            if elapsed > effective_timeout:
                self._km.interrupt_kernel()

                return f'[ERROR] Execution timed out after {effective_timeout} seconds'

            try:
                msg = client.get_iopub_msg(timeout=1.0)

            except queue.Empty:
                continue

            if msg.get('parent_header', {}).get('msg_id') != msg_id:
                continue

            msg_type = msg.get('msg_type')
            content = msg.get('content', {})

            if msg_type == 'stream':
                text = content.get('text', '')

                if content.get('name') == 'stdout':
                    stdout_parts.append(text)

                else:
                    stderr_parts.append(text)

            elif msg_type == 'error':
                traceback_list = content.get('traceback', [])

                stderr_parts.append(self._format_error(traceback_list))

            elif msg_type in {'execute_result', 'display_data'}:
                data = content.get('data', {})
                text = data.get('text/plain')

                if text:
                    stdout_parts.append(text if text.endswith('\n') else f'{text}\n')

            elif msg_type == 'status':
                if content.get('execution_state') == 'idle':
                    break

        stdout = ''.join(stdout_parts)
        stderr = ''.join(stderr_parts)

        if stderr:
            return f'{stdout.rstrip()}\n{stderr}' if stdout else stderr

        return stdout if stdout.strip() else '[WARN] No output. Use print() to see results.'

    def close(self):

        with contextlib.suppress(Exception):
            if self._client:
                self._client.stop_channels()

        if self._owns_kernel and self._km is not None:
            with contextlib.suppress(Exception):
                self._km.shutdown_kernel(now=True)

            with contextlib.suppress(Exception):
                self._km.cleanup_resources()

    def reset(self):

        self.execute('%reset -f')
        self.execute('import gc; gc.collect()')

        self.execute(
            'import math\n'
            'import sympy\n'
            'import itertools\n'
            'import collections\n'
            'import numpy as np\n'
        )

    def __del__(self):

        self.close()

In [13]:
class AIMO3Tool:

    def __init__(self, local_jupyter_timeout: float, tool_prompt: str, sandbox=None):

        self._local_jupyter_timeout = local_jupyter_timeout
        self._tool_prompt = tool_prompt
        self._jupyter_session = sandbox
        
        self._owns_session = sandbox is None
        
        self._execution_lock = threading.Lock()
        self._init_lock = threading.Lock()

    def _ensure_session(self):

        if self._jupyter_session is None:
            with self._init_lock:
                if self._jupyter_session is None:
                    self._jupyter_session = AIMO3Sandbox(timeout=self._local_jupyter_timeout)

    def _ensure_last_print(self, code: str) -> str:

        lines = code.strip().split('\n')

        if not lines:
            return code

        last_line = lines[-1].strip()

        if 'print' in last_line or 'import' in last_line:
            return code

        if not last_line:
            return code

        if last_line.startswith('#'):
            return code

        lines[-1] = 'print(' + last_line + ')'

        return '\n'.join(lines)

    @property
    def instruction(self) -> str:

        return self._tool_prompt

    @property
    def tool_config(self) -> ToolNamespaceConfig:

        return ToolNamespaceConfig(
            name='python', 
            description=self.instruction, 
            tools=[]
        )

    def _make_response(self, output: str, channel: str | None = None) -> Message:

        content = TextContent(text=output)
        author = Author(role=Role.TOOL, name='python')
        message = Message(author=author, content=[content]).with_recipient('assistant')

        if channel:
            message = message.with_channel(channel)

        return message

    def process_sync_plus(self, message: Message) -> list[Message]:

        self._ensure_session()
        raw_script = message.content[0].text
        final_script = self._ensure_last_print(raw_script)

        with self._execution_lock:
            try:
                output = self._jupyter_session.execute(final_script)

            except TimeoutError as exc:
                output = f'[ERROR] {exc}'

        return [self._make_response(output, channel=message.channel)]

    def close(self):

        if self._jupyter_session is not None:
            if self._owns_session:
                self._jupyter_session.close()

            self._jupyter_session = None

    def __del__(self):

        self.close()

In [ ]:
class AIMO3Solver:

    def __init__(self, cfg, port: int = 8000):

        self.cfg = cfg
        self.port = port
        self.base_url = f'http://0.0.0.0:{port}/v1'
        self.api_key = 'sk-local'
        self.template = AIMO3Template()
        self.encoding = load_harmony_encoding(HarmonyEncodingName.HARMONY_GPT_OSS)
        self.stop_token_ids = self.encoding.stop_tokens_for_assistant_actions()

        self._preload_model_weights()
        
        self.server_process = self._start_server()

        self.client = OpenAI(
            base_url=self.base_url, 
            api_key=self.api_key, 
            timeout=self.cfg.session_timeout
        )

        self._wait_for_server()
        self._initialize_kernels()

        self.notebook_start_time = time.time()
        self.problems_remaining = 50
        self.time_bank = 0.0

    def _preload_model_weights(self) -> None:

        print(f'Loading model weights from {self.cfg.model_path} into OS Page Cache...')
        start_time = time.time()
        
        files_to_load = []
        total_size = 0

        for root, _, files in os.walk(self.cfg.model_path):
            for file_name in files:
                file_path = os.path.join(root, file_name)

                if os.path.isfile(file_path):
                    files_to_load.append(file_path)
                    total_size += os.path.getsize(file_path)

        def _read_file(path: str) -> None:

            with open(path, 'rb') as file_object:
                while file_object.read(1024 * 1024 * 1024):
                    pass

        with ThreadPoolExecutor(max_workers=self.cfg.workers) as executor:
            list(executor.map(_read_file, files_to_load))

        elapsed = time.time() - start_time
        print(f'Processed {len(files_to_load)} files ({total_size / 1e9:.2f} GB) in {elapsed:.2f} seconds.\n')

    def _start_server(self) -> subprocess.Popen:

        cmd = [
            sys.executable, 
            '-m', 
            'vllm.entrypoints.openai.api_server', 
            '--seed', 
            str(self.cfg.seed), 
            '--model', 
            self.cfg.model_path, 
            '--served-model-name', 
            self.cfg.served_model_name, 
            '--tensor-parallel-size', 
            '1', 
            '--max-num-seqs', 
            str(self.cfg.batch_size), 
            '--gpu-memory-utilization', 
            str(self.cfg.gpu_memory_utilization), 
            '--host', 
            '0.0.0.0', 
            '--port', 
            str(self.port), 
            '--dtype', 
            self.cfg.dtype, 
            '--kv-cache-dtype', 
            self.cfg.kv_cache_dtype, 
            '--max-model-len', 
            str(self.cfg.context_tokens), 
            '--stream-interval', 
            str(self.cfg.stream_interval), 
            '--async-scheduling', 
            '--enable-prefix-caching'
        ]

        self.log_file = open('vllm_server.log', 'w')

        return subprocess.Popen(
            cmd, 
            stdout=self.log_file, 
            stderr=subprocess.STDOUT, 
            start_new_session=True
        )

    def _wait_for_server(self):

        print('Waiting for vLLM server...')
        start_time = time.time()

        for _ in range(self.cfg.server_timeout):
            return_code = self.server_process.poll()

            if return_code is not None:
                self.log_file.flush()

                with open('vllm_server.log', 'r') as log_file:
                    logs = log_file.read()

                raise RuntimeError(f'Server died with code {return_code}. Full logs:\n{logs}\n')

            try:
                self.client.models.list()
                elapsed = time.time() - start_time
                print(f'Server is ready (took {elapsed:.2f} seconds).\n')

                return

            except Exception:
                time.sleep(1)

        raise RuntimeError('Server failed to start (timeout).\n')

    def _initialize_kernels(self) -> None:

        print(f'Initializing {self.cfg.workers} persistent Jupyter kernels...')
        start_time = time.time()

        self.sandbox_pool = queue.Queue()

        def _create_sandbox():
            
            return AIMO3Sandbox(timeout=self.cfg.jupyter_timeout)

        with ThreadPoolExecutor(max_workers=self.cfg.workers) as executor:
            futures = [executor.submit(_create_sandbox) for _ in range(self.cfg.workers)]

            for future in as_completed(futures):
                self.sandbox_pool.put(future.result())

        elapsed = time.time() - start_time
        print(f'Kernels initialized in {elapsed:.2f} seconds.\n')

    def _scan_for_answer(self, text: str) -> int | None:

        pattern = r'\\boxed\s*\{\s*([0-9,]+)\s*\}'
        matches = re.findall(pattern, text)

        if matches:
            try:
                clean_value = matches[-1].replace(',', '')
                value = int(clean_value)

                if 0 <= value <= 99999:
                    return value

            except ValueError:
                pass

        return None

    def _process_attempt(
        self, 
        problem: str, 
        system_prompt: str, 
        attempt_index: int, 
        stop_event: threading.Event, 
        deadline: float
    ) -> dict:

        if stop_event.is_set() or time.time() > deadline:
            return {
                'Attempt': attempt_index + 1, 
                'Answer': None, 
                'Python Calls': 0, 
                'Python Errors': 0, 
                'Response Length': 0
            }

        local_tool = None
        sandbox = None
        python_calls = 0
        python_errors = 0
        total_tokens = 0
        final_answer = None

        attempt_seed = int(math.pow(self.cfg.seed + attempt_index, 2))

        try:
            sandbox = self.sandbox_pool.get(timeout=self.cfg.sandbox_timeout)

            local_tool = AIMO3Tool(
                local_jupyter_timeout=self.cfg.jupyter_timeout, 
                tool_prompt=self.cfg.tool_prompt, 
                sandbox=sandbox
            )

            encoding = self.encoding
            messages = self.template.apply_chat_template(
                system_prompt, 
                problem, 
                local_tool.tool_config
            )

            conversation = Conversation.from_messages(messages)

            for _ in range(self.cfg.turns):
                if stop_event.is_set() or time.time() > deadline:
                    break

                prompt_ids = encoding.render_conversation_for_completion(conversation, Role.ASSISTANT)
                max_tokens = self.cfg.context_tokens - len(prompt_ids)

                if max_tokens < self.cfg.buffer_tokens:
                    break

                # Create stream request with error handling
                stream = None
                try:
                    stream = self.client.completions.create(
                        model=self.cfg.served_model_name, 
                        temperature=self.cfg.temperature, 
                        max_tokens=max_tokens, 
                        prompt=prompt_ids, 
                        seed=attempt_seed, 
                        stream=True, 
                        extra_body={
                            'min_p': self.cfg.min_p, 
                            'stop_token_ids': self.stop_token_ids, 
                            'return_token_ids': True
                        },
                        timeout=max(0,deadline - time.time()),
                    )
                except Exception as e:
                    print(f"Failed to create completion stream: {e}")
                    break
    
                if stream is None:
                    continue

                try:
                    token_buffer = []
                    text_chunks = []

                    for chunk in stream:
                        if stop_event.is_set() or time.time() > deadline:
                            break

                        new_tokens = chunk.choices[0].token_ids
                        new_text = chunk.choices[0].text

                        if new_tokens:
                            token_buffer.extend(new_tokens)
                            total_tokens += len(new_tokens)
                            text_chunks.append(new_text)

                        if '}' in new_text:
                            search_text = ''.join(text_chunks[-self.cfg.search_tokens:])
                            answer = self._scan_for_answer(search_text)

                            if answer is not None:
                                final_answer = answer
                                break

                finally:
                    stream.close()

                if final_answer is not None:
                    break

                if not token_buffer:
                    break

                new_messages = encoding.parse_messages_from_completion_tokens(token_buffer, Role.ASSISTANT)
                conversation.messages.extend(new_messages)
                last_message = new_messages[-1]

                if last_message.channel == 'final':
                    answer_text = last_message.content[0].text
                    final_answer = self._scan_for_answer(answer_text)
                    break

                if last_message.recipient == 'python':
                    python_calls += 1
                    tool_responses = local_tool.process_sync_plus(last_message)

                    response_text = tool_responses[0].content[0].text

                    if response_text.startswith('[ERROR]') or 'Traceback' in response_text or 'Error:' in response_text:
                        python_errors += 1

                    conversation.messages.extend(tool_responses)

        except Exception as exc:
            python_errors += 1

        finally:
            if local_tool is not None:
                local_tool.close()

            if sandbox is not None:
                sandbox.reset()
                self.sandbox_pool.put(sandbox)

        return {
            'Attempt': attempt_index + 1, 
            'Response Length': total_tokens, 
            'Python Calls': python_calls, 
            'Python Errors': python_errors, 
            'Answer': final_answer
        }

    def _select_answer(self, detailed_results: list) -> int:

        stats = defaultdict(lambda: {'votes': 0, 'calls': 0})

        for result in detailed_results:
            answer = result['Answer']

            if answer is not None:
                stats[answer]['votes'] += 1
                stats[answer]['calls'] += result['Python Calls']

        sorted_stats = sorted(
            stats.items(), 
            key=lambda item: (item[1]['votes'], item[1]['calls']), 
            reverse=True
        )

        vote_data = []

        for answer, data in sorted_stats:
            vote_data.append((answer, data['votes'], data['calls']))

        vote_dataframe = pd.DataFrame(vote_data, columns=['Answer', 'Votes', 'Calls'])
        display(vote_dataframe)

        final_answer = sorted_stats[0][0]
        final_votes = sorted_stats[0][1]['votes']
        final_calls = sorted_stats[0][1]['calls']

        print(f'\nFinal Result: {final_answer} | Votes: {final_votes} | Calls: {final_calls}\n')

        return final_answer

    def solve_problem(self, problem: str) -> int:
        
        problem_start_time = time.time()
        print(f'\nProblem: {problem}\n')

        user_input = f'{problem} {self.cfg.preference_prompt}'

        # === Budget calculation with time banking ===
        elapsed_global = time.time() - self.notebook_start_time
        time_left = self.cfg.notebook_limit - elapsed_global
        problems_left_others = max(0, self.problems_remaining - 1)
        reserved_time = problems_left_others * self.cfg.base_problem_timeout

        budget = time_left - reserved_time
        budget = min(budget, self.cfg.high_problem_timeout)
        budget = max(budget, self.cfg.base_problem_timeout)

        bank_bonus = min(self.time_bank, self.cfg.max_bank_bonus)
        budget += bank_bonus
        self.time_bank -= bank_bonus

        deadline = time.time() + budget

        print(f'Budget: {budget:.2f}s (bank bonus: {bank_bonus:.2f}s) | Time bank: {self.time_bank:.2f}s\n')

        # === ROUND 1: Standard attempts ===
        tasks = []
        for attempt_index in range(self.cfg.attempts):
            tasks.append((self.cfg.system_prompt, attempt_index))

        detailed_results = []
        valid_answers = []
        stop_event = threading.Event()
        early_stopped = False

        executor = ThreadPoolExecutor(max_workers=self.cfg.workers)

        try:
            futures = []

            for (system_prompt, attempt_index) in tasks:
                future = executor.submit(
                    self._process_attempt, 
                    user_input, 
                    system_prompt, 
                    attempt_index, 
                    stop_event, 
                    deadline
                )
                futures.append(future)

            for future in as_completed(futures):
                try:
                    result = future.result()
                    detailed_results.append(result)

                    if result['Answer'] is not None:
                        valid_answers.append(result['Answer'])

                    counts = Counter(valid_answers).most_common(1)

                    if counts and counts[0][1] >= self.cfg.early_stop:
                        stop_event.set()
                        early_stopped = True

                        for f in futures:
                            f.cancel()

                        break

                except Exception as exc:
                    print(f'Future failed: {exc}')
                    continue

        finally:
            executor.shutdown(wait=False, cancel_futures=True)

        # === Check consensus for retry ===
        counts = Counter(valid_answers).most_common(1)
        best_votes = counts[0][1] if counts else 0
        time_remaining = deadline - time.time()

        # === ROUND 2: Adaptive retry if low consensus ===
        if not early_stopped and best_votes < self.cfg.min_consensus and time_remaining > self.cfg.min_retry_time:
            candidate_answers = list(set(valid_answers))
            candidates_str = ', '.join(str(a) for a in candidate_answers) if candidate_answers else 'none'

            retry_input = (
                f'{problem} {self.cfg.preference_prompt} '
                f'Previous attempts suggested these answers: {candidates_str}. '
                f'Carefully verify using a different approach.'
            )

            print(f'[RETRY] Low consensus ({best_votes} votes). '
                  f'Running {self.cfg.retry_attempts} more attempts '
                  f'({time_remaining:.0f}s remaining)...\n')

            stop_event2 = threading.Event()
            executor2 = ThreadPoolExecutor(max_workers=self.cfg.workers)

            try:
                futures2 = []

                for attempt_index in range(self.cfg.retry_attempts):
                    future = executor2.submit(
                        self._process_attempt,
                        retry_input,
                        self.cfg.retry_system_prompt,
                        self.cfg.attempts + attempt_index,
                        stop_event2,
                        deadline
                    )
                    futures2.append(future)

                for future in as_completed(futures2):
                    try:
                        result = future.result()
                        detailed_results.append(result)

                        if result['Answer'] is not None:
                            valid_answers.append(result['Answer'])

                    except Exception as exc:
                        print(f'Retry future failed: {exc}')
                        continue

            finally:
                executor2.shutdown(wait=False, cancel_futures=True)

        self.problems_remaining = max(0, self.problems_remaining - 1)

        # === Time banking: deposit unused time ===
        used_time = time.time() - problem_start_time
        unused_time = max(0.0, budget - used_time)
        self.time_bank = max(0.0, self.time_bank + unused_time)

        print(f"[Budget]: {budget:.2f}s\n")
        print(f"[Inference] Took {used_time:.2f}s\n")
        print(f"[Saved]: {unused_time:.2f}s | [Time bank]: {self.time_bank:.2f}s\n")

        if detailed_results:
            results_dataframe = pd.DataFrame(detailed_results)
            results_dataframe['Answer'] = results_dataframe['Answer'].astype('Int64')
            display(results_dataframe)

        if not valid_answers:
            print('\nResult: 0\n')
            return 0

        return self._select_answer(detailed_results)

    def __del__(self):

        if hasattr(self, 'server_process'):
            self.server_process.terminate()
            self.server_process.wait()

        if hasattr(self, 'log_file'):
            self.log_file.close()

        if hasattr(self, 'sandbox_pool'):
            while not self.sandbox_pool.empty():
                try:
                    sb = self.sandbox_pool.get_nowait()
                    sb.close()

                except Exception:
                    pass

In [15]:
solver = AIMO3Solver(CFG)

Loading model weights from /kaggle/input/gpt-oss-120b/transformers/default/1 into OS Page Cache...
Processed 26 files (65.28 GB) in 77.35 seconds.

Waiting for vLLM server...
Server is ready (took 123.87 seconds).

Initializing 16 persistent Jupyter kernels...
Kernels initialized in 3.01 seconds.



In [16]:
def predict(id_: pl.DataFrame, question: pl.DataFrame, answer: Optional[pl.DataFrame] = None) -> pl.DataFrame:
    global correct_count, total_count, predictions
    
    question_id = id_.item(0)
    question_text = question.item(0)
    
    print("------")
    print(f"ID: {question_id}")
    print(f"Question: {question_text[:200]}...")
    
    final_answer = solver.solve_problem(question_text)
    predictions[question_id] = final_answer

    # Check accuracy if ground truth available
    total_count += 1
    if question_id in ground_truth:
        gt = ground_truth[question_id]
        is_correct = (final_answer == gt)
        if is_correct:
            correct_count += 1
        status = "✅" if is_correct else "❌"
        print(f"Answer: {final_answer} | Ground Truth: {gt} | {status}")
        print(f"📊 Running Accuracy: {correct_count}/{total_count} ({100*correct_count/total_count:.1f}%)")
    else:
        print(f"Answer: {final_answer}")
    
    print("------\n")
    
    return pl.DataFrame({'id': question_id, 'answer': final_answer})

In [17]:
# Load reference data and keep ground truth for accuracy calculation
df = pd.read_csv(
    "/kaggle/input/ai-mathematical-olympiad-progress-prize-3/reference.csv"
)

# Store ground truth answers for accuracy calculation (only in local mode)
ground_truth = dict(zip(df["id"], df["answer"])) if "answer" in df.columns else {}

# Create input file without answers
df.drop("answer", axis=1, errors="ignore").to_csv("reference.csv", index=False)

# Track predictions for accuracy calculation
predictions = {}
correct_count = 0
total_count = 0

In [18]:
inference_server = kaggle_evaluation.aimo_3_inference_server.AIMO3InferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
    
else:
    inference_server.run_local_gateway(("reference.csv",))
    #inference_server.run_local_gateway(
    #    ('/kaggle/input/ai-mathematical-olympiad-progress-prize-3/test.csv',)
    #)

------
ID: 42d360
Question: On a blackboard, Ken starts off by writing a positive integer $n$ and then applies the following move until he first reaches $1$. Given that the number on the board is $m$, he chooses a base $b$, wher...

Problem: On a blackboard, Ken starts off by writing a positive integer $n$ and then applies the following move until he first reaches $1$. Given that the number on the board is $m$, he chooses a base $b$, where $2 \leq b \leq m$, and considers the unique base-$b$ representation of $m$,
\begin{equation*}
    m = \sum_{k = 0}^\infty a_k \cdot b^k
\end{equation*}
where $a_k$ are non-negative integers and $0 \leq a_k < b$ for each $k$. Ken then erases $m$ on the blackboard and replaces it with $\sum\limits_{k = 0}^\infty a_k$.

Across all choices of $1 \leq n \leq 10^{10^5}$, the largest possible number of moves Ken could make is $M$. What is the remainder when $M$ is divided by $10^{5}$?

Budget: 900.00 seconds | Deadline: 1770313662.08

🐍 Executing Python cod

,Attempt,Response Length,Python Calls,Python Errors,Answer
0,1,6315,12,0,32193
1,7,6393,7,0,32193
2,8,6841,6,0,32193
3,6,6894,6,0,32193


,Answer,Votes,Calls
0,32193,4,31



Final Result: 32193 | Votes: 4 | Calls: 31

Answer: 32193 | Ground Truth: 32193 | ✅
📊 Running Accuracy: 1/1 (100.0%)
------

------
ID: 0e644e
Question: Let $ABC$ be an acute-angled triangle with integer side lengths and $AB<AC$. Points $D$ and $E$ lie on segments $BC$ and $AC$, respectively, such that $AD=AE=AB$. Line $DE$ intersects $AB$ at $X$. Cir...

Problem: Let $ABC$ be an acute-angled triangle with integer side lengths and $AB<AC$. Points $D$ and $E$ lie on segments $BC$ and $AC$, respectively, such that $AD=AE=AB$. Line $DE$ intersects $AB$ at $X$. Circles $BXD$ and $CED$ intersect for the second time at $Y \neq D$. Suppose that $Y$ lies on line $AD$. There is a unique such triangle with minimal perimeter. This triangle has side lengths $a=BC$, $b=CA$, and $c=AB$. Find the remainder when $abc$ is divided by $10^{5}$.

Budget: 900.00 seconds | Deadline: 1770313726.38

🐍 Executing Python code...
🐍 Executing Python code...
🐍 Executing Python code...
🐍 Executing Python code...
🐍 

,Attempt,Response Length,Python Calls,Python Errors,Answer
0,3,11313,14,0,336
1,8,12270,17,2,336
2,7,12484,6,0,2688
3,1,12991,9,0,336
4,6,13372,14,1,336


...

,Answer,Votes,Calls
0,57447,4,69



Final Result: 57447 | Votes: 4 | Calls: 69

Answer: 57447 | Ground Truth: 57447 | ✅
📊 Running Accuracy: 8/9 (88.9%)
------

------
ID: 9c1c5f
Question: Let $f \colon \mathbb{Z}_{\geq 1} \to \mathbb{Z}_{\geq 1}$ be a function such that for all positive integers $m$ and $n$, 
\begin{equation*}
    f(m) + f(n) = f(m + n + mn).
\end{equation*}
Across all...

Problem: Let $f \colon \mathbb{Z}_{\geq 1} \to \mathbb{Z}_{\geq 1}$ be a function such that for all positive integers $m$ and $n$, 
\begin{equation*}
    f(m) + f(n) = f(m + n + mn).
\end{equation*}
Across all functions $f$ such that $f(n) \leq 1000$ for all $n \leq 1000$, how many different values can $f(2024)$ take?

Budget: 900.00 seconds | Deadline: 1770315405.76

🐍 Executing Python code...
🐍 Executing Python code...
🐍 Executing Python code...
🐍 Executing Python code...
🐍 Executing Python code...
🐍 Executing Python code...
🐍 Executing Python code...
🐍 Executing Python code...
🐍 Executing Python code...
🐍 Executing Python code...
🐍

,Attempt,Response Length,Python Calls,Python Errors,Answer
0,8,8556,10,0,580
1,7,8310,9,0,580
2,3,8966,9,0,580
3,2,9193,10,0,580


,Answer,Votes,Calls
0,580,4,38



Final Result: 580 | Votes: 4 | Calls: 38

Answer: 580 | Ground Truth: 580 | ✅
📊 Running Accuracy: 9/10 (90.0%)
------

